## Imports

In [ ]:
from pathlib import Path
from PIL import Image
import os
import json
from typing import Literal
import re
import shutil
import random

## Configuration

### Selecting folders

In [ ]:
target_folder = Path('./website')
title = 'GALLERY'
image_folder = Path('./images')
background_folder = Path('./backgrounds')
video_folder = Path('./videos')

In [ ]:
try:
  import config
  target_folder = config.target_folder
  image_folder = config.input_image_folder
  background_folder = config.background_folder
  video_folder = config.video_folder
except:
  print("config.py not found, using default values.")

In [ ]:
try: # this only works for windows
  if input('Wanna select folders? (empty for no)'):
    from tkinter.filedialog import askdirectory
    target_folder = askdirectory(title='Target Folder')
    image_folder = askdirectory(title='Images Folder')
    background_folder = askdirectory(title='Backgrounds Folder')
    video_folder = askdirectory(title='Videos Folder')
except Exception:
  pass

In [ ]:
print(f"Target Folder: {target_folder}")
print(f"Images Folder: {image_folder}")
print(f"Backgrounds Folder: {background_folder}")
print(f"Videos Folder: {video_folder}")

### Fileendings

In [ ]:
VIDEO_FILE_ENDINGS = ['mp4', 'mov', 'webm', 'avi']
IMAGE_FILE_ENDINGS = ['jpg', 'jpeg', 'png', 'gif', 'bmp', 'webp']

### Targetstructure

Do not change

In [ ]:
ASSET_IMAGES_FOLDER_NAME = 'images'
ASSET_VIDEOS_FOLDER_NAME = 'videos'
ASSET_BACKGROUND_FOLDER_NAME = 'backgrounds'

In [ ]:
asset_folder = os.path.join(target_folder, 'assets')
data_folder = os.path.join(asset_folder, 'data')
images_folder = os.path.join(asset_folder, ASSET_IMAGES_FOLDER_NAME)
backgrounds_folder = os.path.join(asset_folder, ASSET_BACKGROUND_FOLDER_NAME)
videos_folder = os.path.join(asset_folder, ASSET_VIDEOS_FOLDER_NAME)

## Copying and Transforming Images

In [ ]:
# target folder
for f in os.listdir(target_folder):
  if os.path.isdir(os.path.join(target_folder, f)):
    shutil.rmtree(os.path.join(target_folder, f))

In [ ]:
# Create Folders if they don't exist yet

if not os.path.exists(target_folder):
  os.mkdir(target_folder)
if not os.path.exists(asset_folder):
  os.mkdir(asset_folder)
if not os.path.exists(images_folder):
  os.mkdir(images_folder)
if not os.path.exists(backgrounds_folder):
  os.mkdir(backgrounds_folder)
if not os.path.exists(videos_folder):
  os.mkdir(videos_folder)

In [ ]:
# helper functions

def get_image_name(idx:int, ending:str='jpg'):
  return f'img_{idx}.{ending}'

def get_video_name(idx:int, file_extension:str='mov'):
  return f'vid_{idx}.{file_extension}'

def strip_exif_and_copy(file_path:Path, output_file_path:Path):
  image = Image.open(file_path)

  data = list(image.getdata())
  image_without_exif = Image.new(image.mode, image.size)
  image_without_exif.putdata(data)
    
  image_without_exif.save(output_file_path)
  

  # as a good practice, close the file handler after saving the image.
  image_without_exif.close()

  
def transform_all_images(image_folder:Path, output_folder:Path):
  image_index = 0
  original_file_names = []
  for root, dirs, files in os.walk(image_folder):
    for file in files:
      for ending in IMAGE_FILE_ENDINGS:
        if file.endswith(ending):
          image_index += 1
          strip_exif_and_copy(os.path.join(root, file), os.path.join(output_folder, get_image_name(image_index, ending)))
          original_file_names.append(file)
          break
  return original_file_names

def copy_videos(video_folder:Path, output_folder:Path):
  video_index = 0
  original_video_file_names = []
  for root, dirs, files in os.walk(video_folder):
    for file in files:
      for ending in VIDEO_FILE_ENDINGS:
        if file.endswith(ending):
          video_index += 1
          shutil.copyfile(os.path.join(root, file), os.path.join(output_folder, get_video_name(video_index, ending)))
          original_video_file_names.append(file)
          break
  return original_video_file_names

In [ ]:
# shifting files around
original_image_file_names = transform_all_images(image_folder, images_folder)
original_background_file_names = transform_all_images(background_folder, backgrounds_folder)
original_video_file_names = copy_videos(video_folder, videos_folder)

In [ ]:
video_folder

## Create Metadata File

In [ ]:
# create data folder if not exists

if not os.path.exists(data_folder):
  os.makedirs(data_folder)

In [ ]:
# helper functions

def create_all_media_json():
  # collect filenames
  detected_image_file_names = [f for f in os.listdir(images_folder) if os.path.isfile(os.path.join(images_folder, f)) and any(f.endswith(ending) for ending in IMAGE_FILE_ENDINGS)]
  detected_video_file_names = [f for f in os.listdir(videos_folder) if os.path.isfile(os.path.join(videos_folder, f)) and any(f.endswith(ending) for ending in VIDEO_FILE_ENDINGS)]
  detected_background_file_names = [f for f in os.listdir(backgrounds_folder) if os.path.isfile(os.path.join(backgrounds_folder, f)) and any(f.endswith(ending) for ending in IMAGE_FILE_ENDINGS)]
  # add foldernames
  detected_image_file_names = [{'src': './assets/images/' + f, 'type':'image'} for f in detected_image_file_names]
  detected_video_file_names = [{'src': './assets/videos/' + f, 'type':'video'} for f in detected_video_file_names]
  detected_background_file_names = [{'src': './assets/backgrounds/' + f, 'type':'image'} for f in detected_background_file_names]
  media_json = {
    'medias': detected_image_file_names + detected_video_file_names, # TODO Try adding sorting
    'backgrounds': detected_background_file_names
  }
  with open(os.path.join(data_folder, 'media.json'), 'w') as f:
    json.dump(media_json, f)

In [ ]:
create_all_media_json()

## Copy html files

In [ ]:
html_folder = Path('./html')

# Copy all files from html_folder to target_folder
for item in html_folder.iterdir():
  if item.is_file():
    shutil.copy(item, target_folder)